## To-do

+ 对象：
    + 五绝（二联，四句）
    + 五律（四联，八句）
+ 原始诗歌文本预处理（分字）
+ 采用《平水韵》韵书判断平仄（问题：多音字）
+ 判断符合格律：
    + 五绝（律）平起首句入韵
    + 五绝（律）仄起首句入韵
    + 五绝（律）平起首句不入韵
    + 五绝（律）仄起首句不入韵
+ 判断押韵
    + 失粘、失对、三平、孤平
+ 编码使符合 TEI Verse 格式
    + 解读 TEI Verse 标签******

In [1]:
# encoding=utf-8
import jieba
import nltk
import json
import re

In [2]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [9]:
with open("data/rhymebooks.json") as output:
    rhymes = json.load(output)
with open("data/TC2SC.json") as output:
    T2C = json.load(output)
with open("data/kangxi.json") as output:
    Dict = json.load(output)
with open("data/metric.name") as output:
    metric_names = [line.strip() for line in output]

In [10]:
print(metric_names)

['一东', '二冬', '三江', '四支', '五微', '六鱼', '七虞', '八齐', '九佳', '十灰', '十一真', '十二文', '十三元', '十四寒', '十五删', '一先', '二萧', '三肴', '四豪', '五歌', '六麻', '七阳', '八庚', '九青', '十蒸', '十一尤', '十二侵', '十三覃', '十四盐', '十五咸', '一董', '二肿', '三讲', '四纸', '五尾', '六语', '七麌', '八荠', '九蟹', '十贿', '十一轸', '十二吻', '十三阮', '十四旱', '十五潸', '十六铣', '十七筱', '十八巧', '十九皓', '二十哿', '二十一马', '二十二养', '二十三梗', '二十四迥', '二十五有', '二十六寝', '二十七感', '二十八琰', '二十九豏', '一送', '二宋', '三绛', '四寘', '五未', '六御', '七遇', '八霁', '九泰', '十卦', '十一队', '十二震', '十三问', '十四愿', '十五翰', '十六谏', '十七霰', '十八啸', '十九效', '二十号', '二十一个', '二十二祃', '二十三漾', '二十四敬', '二十五径', '二十六宥', '二十七沁', '二十八勘', '二十九艳', '三十陷', '一屋', '二沃', '三觉', '四质', '五物', '六月', '七曷', '八黠', '九屑', '十药', '十一陌', '十二锡', '十三职', '十四缉', '十五合', '十六叶', '十七洽']


In [11]:
# 《平水韵》有入声，《中华新韵》无。
# 诗用《平水韵》，词用《词林正韵》
P_PING_unflattened = rhymes['平水韵'][0]
Z_PING_unflattened = rhymes['平水韵'][1]
P_PING = flatten(rhymes['平水韵'][0])
Z_PING = flatten(rhymes['平水韵'][1])
# P_ZHXY = flatten(rhymes['中华新韵'][0])
# Z_ZHXY = flatten(rhymes['中华新韵'][1])

In [12]:
raw1 = "白日依山尽，黄河入海流。欲穷千里目，更上一层楼。"
raw2 = "迟日江山丽，春风花草香。 泥融飞燕子， 沙暖睡鸳鸯。"
raw3 = "鸣筝金粟柱, 素手玉房前, 欲得周郎顾, 时时误拂弦"
raw4 = "花明绮陌春，柳拂御沟新。为报辽阳客，流芳不待人。"
raw5 = "北斗七星高，哥舒夜带刀。至今窥牧马，不敢过临洮。"
raw6 = "离离原上草，一岁一枯荣。野火烧不尽，春风吹又生。"

In [13]:
raw7 = "泠泠七弦上，静听松风寒。古调虽自爱，今人多不弹。"

In [13]:
keywords = jieba.cut('欲穷千里目，更上一层楼。', cut_all=False)
" ".join(keywords)

'欲穷千里目 ， 更上一层楼 。'

In [15]:
jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持
strs = raw1
for str in strs:
    seg_list = jieba.cut(str,use_paddle=True) # 使用paddle模式
    print("Paddle Mode: " + '/'.join(list(seg_list)))

seg_list = jieba.cut(raw1, cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut(raw1, cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut(raw1)  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search(raw1)  # 搜索引擎模式
print(", ".join(seg_list))

Paddle enabled successfully......


Paddle Mode: 白
Paddle Mode: 日
Paddle Mode: 依
Paddle Mode: 山
Paddle Mode: 尽
Paddle Mode: ，
Paddle Mode: 黄
Paddle Mode: 河
Paddle Mode: 入
Paddle Mode: 海
Paddle Mode: 流
Paddle Mode: 。
Paddle Mode: 欲
Paddle Mode: 穷
Paddle Mode: 千
Paddle Mode: 里
Paddle Mode: 目
Paddle Mode: ，
Paddle Mode: 更
Paddle Mode: 上
Paddle Mode: 一
Paddle Mode: 层
Paddle Mode: 楼
Paddle Mode: 。
Full Mode: 白日/ 依/ 山/ 尽/ ，/ 黄河/ 河入海/ 入海/ 入海流/ 海流/ 。/ 欲穷千里/ 欲穷千里目/ 千里/ 千里目/ ，/ 更上一层/ 更上一层楼/ 一层/ 一层楼/ 层楼/ 。
Default Mode: 白日/ 依山/ 尽/ ，/ 黄河/ 入海流/ 。/ 欲穷千里目/ ，/ 更上一层楼/ 。
白日, 依山, 尽, ，, 黄河, 入海流, 。, 欲穷千里目, ，, 更上一层楼, 。
白日, 依山, 尽, ，, 黄河, 入海, 海流, 入海流, 。, 千里, 千里目, 欲穷千里目, ，, 一层, 层楼, 一层楼, 更上一层楼, 。


In [14]:
def preprocess(raw):
    clean = re.sub(r'[，。, .]', "", raw)
    tokens = [token for token in clean if token]
    tones  = []
    for t in tokens:
        if t in P_PING:
            if t in Z_PING:
                tones.append('1/0') 
            else:
                tones.append('1') 
        else:
            tones.append('0')
    return tokens,tones

In [15]:
metrics = []

# 五绝平起首句入韵
metrics.append([
    '1','1','1/0','0','1',
    '1/0','0','0','1','1',
    '1/0','0','1','1','0',
    '1','1','1/0','0','1'])
# 五绝平起首句不入韵
metrics.append([
    '1/0','1','1','0','0',
    '1/0','0','0','1','1',
    '1/0','0','1','1','0',
    '1','1','1/0','0','1'])
# 五绝仄起首句入韵
metrics.append([
    '1/0','0','0','1','1',
    '1','1','1/0','0','1',
    '1/0','1','1','0','0',
    '1/0','0','0','1','1'])
# 五绝仄起首句不入韵
metrics.append([
    '1/0','0','1','1','0',
    '1','1','1/0','0','1',
    '1/0','1','1','0','0',
    '1/0','0','0','1','1'])

In [44]:
def metricsAnalytics(verse, metrics):
    tokens, tones = verse
    print('五绝') 
    count = 0 
    RBOOK = P_PING_unflattened
    if tones[1] == '1':
        print('平起') 
        metric = metrics[:2]
    else:
        print('仄起')
        metric = metrics[2:]
        
    for k,v in enumerate(RBOOK):
        if tokens[9] in v and tokens[19] in v:
            print("韵脚：", metric_names[k])
            if tones[4] == '1':
                num_metrics = 0
                if tokens[4] in v:
                    print('首句入韵')
            else:
                num_metrics = 1
                print('首句仄收，不入韵, 首句白脚')

                for k,(l1,l2) in enumerate(zip(tones,metric[num_metrics])):
                    if l1 == '1/0':
                        print("[检测到多音字",tokens[k],"]")
                    elif l2 == '1/0':
                        pass
                    elif l1 != l2 and k%5 != 0:
                        if l1 == '0':
                            print(tokens[k],"失配,建议：仄")
                        else:
                            print(tokens[k],"失配,建议：平")
                        count += 1
   
    print('失配词数',count)

In [45]:
verse1 = preprocess(raw1)
verse2 = preprocess(raw2)
verse3 = preprocess(raw3)
verse4 = preprocess(raw4)
verse5 = preprocess(raw5) 
verse6 = preprocess(raw6)
verse7 = preprocess(raw7)

In [46]:
metricsAnalytics(verse1, metrics)

五绝
仄起
韵脚： 十一尤
首句仄收，不入韵, 首句白脚
[检测到多音字 更 ]
失配词数 0


In [47]:
metricsAnalytics(verse2, metrics)

五绝
仄起
韵脚： 七阳
首句仄收，不入韵, 首句白脚
[检测到多音字 丽 ]
[检测到多音字 泥 ]
[检测到多音字 燕 ]
失配词数 0


In [48]:
metricsAnalytics(verse3, metrics)

五绝
平起
韵脚： 一先
首句仄收，不入韵, 首句白脚
失配词数 0


In [49]:
metricsAnalytics(verse4, metrics)

五绝
平起
韵脚： 十一真
首句入韵
失配词数 0


In [50]:
metricsAnalytics(verse5, metrics)

五绝
仄起
韵脚： 四豪
首句入韵
失配词数 0


In [51]:
metricsAnalytics(verse6, metrics)

五绝
平起
韵脚： 八庚
首句仄收，不入韵, 首句白脚
[检测到多音字 不 ]
[检测到多音字 吹 ]
失配词数 0


In [52]:
metricsAnalytics(verse7, metrics)

五绝
平起
韵脚： 十四寒
首句仄收，不入韵, 首句白脚
七 失配,建议：仄
弦 失配,建议：平
[检测到多音字 听 ]
松 失配,建议：平
[检测到多音字 调 ]
自 失配,建议：仄
[检测到多音字 不 ]
[检测到多音字 弹 ]
失配词数 4
